In [1]:
import psycopg2 
import pandas as pd
import threading
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor

### Loading Data

In [2]:
conn = psycopg2.connect(
    host="fit3164db.cxkhqsoitzhb.ap-southeast-2.rds.amazonaws.com", 
    port=5432,
    user="postgres",
    password="fit3164d13edas",
    database = "testdb"
)
conn.autocommit = True
cursor = conn.cursor()

In [3]:
#actuals columns
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'actuals'")
acc_cols = cursor.fetchall()
acc_cols = [item[0] for item in acc_cols]

cursor.execute("select * from actuals")
actual_data = cursor.fetchall()

In [4]:
#forecasts columns
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'forecasts'")
fc_cols = cursor.fetchall()
fc_cols = [item[0] for item in fc_cols]

cursor.execute("select * from forecasts")
forecast_data = cursor.fetchall()

In [5]:
cursor.close()
conn.close()

In [6]:
actuals = pd.DataFrame(actual_data, columns= acc_cols)
forecast = pd.DataFrame(forecast_data, columns=fc_cols)

In [7]:
def convert_dtypes(df, actual=True):
    if actual:
        numeric_cols = ['month','hour', 'type_of_day', 'load', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    else:
        numeric_cols = ['month','hour', 'type_of_day', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    
    date_cols = ['datetime', 'date']
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
        
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    
    return df

actuals = convert_dtypes(actuals)
forecast = convert_dtypes(forecast, False)

In [8]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [9]:
#lags for load 
#num_lags=168
def get_lag_lead(data, num_lags=168, forward_pred=48): 
    cols = ['pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']

    #for load
    for i in range(1, num_lags + 1):
        data[f'load_lag_{i}'] = data['load'].shift(i) 

    #for lagged weather variables
    for c in cols:
        for i in range(1, num_lags+1):
            data[f'{c}_lag_{i}'] = data[c].shift(i)

    for i in range(0, forward_pred+1):    
        data[f'load_lead_{i}'] = data['load'].shift(-i)

    #weather
    for c in cols:
        for i in range(0,forward_pred):
            data[f'{c}_lead_{i}'] = data[c].shift(-i)


    data.dropna(inplace = True)    #drop nulls
    data = data.set_index('datetime')   #set index as datetime
    data = data.asfreq('H')

    data = data.drop(['load', 'pressure', 'wind_speed', 'wind_direction', 'temperature', 'cloud_cover'], axis=1)

    return data

In [10]:
data = pd.merge(actuals[['datetime','load']], forecast, on='datetime')

encode(data, 'hour', 24)
encode(data, 'month', 12)
encode(data, 'type_of_day', 31)

# data = data.set_index('datetime')
data = data.drop(['id', 'date','month', 'hour', 'type_of_day'], axis=1)
data = data.rename(columns = {'type_of_day_sin': 'day_sin', 
                            'type_of_day_cos': 'day_cos'})

In [11]:
data = get_lag_lead(data)

/var/folders/0v/z1x4wq2x2tl7d63st822t7dh0000gn/T/ipykernel_33813/1681627816.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{c}_lead_{i}'] = data[c].shift(-i)
/var/folders/0v/z1x4wq2x2tl7d63st822t7dh0000gn/T/ipykernel_33813/1681627816.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{c}_lead_{i}'] = data[c].shift(-i)
/var/folders/0v/z1x4wq2x2tl7d63st822t7dh0000gn/T/ipykernel_33813/1681627816.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

In [12]:
desired_start_time = '08:00:00'
desired_end_time = '08:00:00'
start_datetime = pd.to_datetime(desired_start_time)
end_datetime = pd.to_datetime(desired_end_time)
data = data.between_time(start_datetime.time(), end_datetime.time())

In [13]:
def split_data(data, test_size=0.1):
    return np.split(data, [int((1 - test_size) * data.shape[0]) + 1])

In [14]:
train, validation = split_data(data)


In [15]:
with open('train_cols.txt', 'w') as f:
    for col in train.columns:
        f.write(col + "\n")

In [16]:
with open('train_cols.txt', 'r') as file:
        for line in file:
            print(line.strip())

hour_sin
hour_cos
month_sin
month_cos
day_sin
day_cos
load_lag_1
pressure_lag_1
cloud_cover_lag_1
temperature_lag_1
wind_speed_lag_1
wind_direction_lag_1
load_lead_0
load_lead_1
load_lead_2
load_lead_3
load_lead_4
load_lead_5
load_lead_6
load_lead_7
load_lead_8
load_lead_9
load_lead_10
load_lead_11
load_lead_12
load_lead_13
load_lead_14
load_lead_15
load_lead_16
load_lead_17
load_lead_18
load_lead_19
load_lead_20
load_lead_21
load_lead_22
load_lead_23
load_lead_24
load_lead_25
load_lead_26
load_lead_27
load_lead_28
load_lead_29
load_lead_30
load_lead_31
load_lead_32
load_lead_33
load_lead_34
load_lead_35
load_lead_36
load_lead_37
load_lead_38
load_lead_39
load_lead_40
load_lead_41
load_lead_42
load_lead_43
load_lead_44
load_lead_45
load_lead_46
load_lead_47
load_lead_48
pressure_lead_0
pressure_lead_1
pressure_lead_2
pressure_lead_3
pressure_lead_4
pressure_lead_5
pressure_lead_6
pressure_lead_7
pressure_lead_8
pressure_lead_9
pressure_lead_10
pressure_lead_11
pressure_lead_12
pressure

original load, pressure, temp, ... are as load_lead_0, pressure_lead_0, temperature_lead_0 etc.

In [17]:
#get specific lead forecast column names from whole df for each model
def get_lead_forecast_names(i):
    cols = ['pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    lead_cols = []
    for c in cols: 
        lead_cols.append(f'{c}_lead_{i}')
    return lead_cols


def x_cols(i):
    # Specify the path to your text file
    file_path = "train_cols.txt"

    # Initialize an empty list to store the words
    word_list = []

    # Open the file and read its contents line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Remove leading and trailing whitespace and append the word to the list
            word_list.append(line.strip())

    word_list += get_lead_forecast_names(i)

    return word_list


def y_cols(i):
    return f'load_lead_{i}'

In [18]:
NUM_OF_MODELS = 48

In [19]:
models = [RandomForestRegressor() for _ in range(NUM_OF_MODELS)]

- get lag load and weather for train and get lead weather for specific hour for train x and valid x
- get load for train y
- get lead load for valid y

In [ ]:
#Multiprocessing version of random forest (9 mins 13secs)

def training_models(i):
    train_x = train[x_cols(i)]
    train_y = train[y_cols(i)]

    current_model = models[i]
    current_model.fit(train_x, train_y)

    return current_model

models = joblib.Parallel(n_jobs=8)(
    joblib.delayed(training_models)(model_index) for model_index in range(NUM_OF_MODELS)
)

In [20]:
#Multithreaded version of random forest (6 mins 2.9 secs)

def training_model(i):
    x_columns = x_cols(i)
    y_columns = y_cols(i)
    
    train_x = train[x_columns]
    train_y = train[y_columns]
 
    current_model = models[i]
    current_model.fit(train_x, train_y)

# Create a list to hold the threads
threads = []

# Iterate through the models and create a thread for each model
for i in range(NUM_OF_MODELS):
    thread = threading.Thread(target=training_model, args=(i,))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

In [ ]:
#Iterative version of random forest (about 61 mins)

for i in range(NUM_OF_MODELS):
    x_columns = x_cols(i)
    y_columns = y_cols(i)
    
    train_x = train[x_columns]
    train_y = train[y_columns]
 
    current_model = models[i]
    current_model.fit(train_x, train_y)

In [ ]:
# Multithreaded version of predicting (9.9 secs)

num_rows = len(validation)
num_columns = NUM_OF_MODELS

predictions = []
for i in range(num_rows):
    p = []
    for j in range(num_columns):
        p.append(0)
    predictions.append(p)
    
for i in range(NUM_OF_MODELS):
    x_columns = x_cols(i)
    valid_x = validation[x_columns]

    current_model = models[i]

    for row in range(len(valid_x)):
        predictions[row][i] = current_model.predict(valid_x.iloc[row, :].values.reshape(1, -1))[0]

In [21]:
# Iterative version of predicting (9.5 secs)

num_rows = len(validation)
num_columns = NUM_OF_MODELS

predictions = []
for i in range(num_rows):
    p = []
    for j in range(num_columns):
        p.append(0)
    predictions.append(p)
    
for i in range(NUM_OF_MODELS):
    x_columns = x_cols(i)
    valid_x = validation[x_columns]

    current_model = models[i]

    for row in range(len(valid_x)):
        predictions[row][i] = current_model.predict(valid_x.iloc[row, :].values.reshape(1, -1))[0]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: Use

In [22]:
predictions = pd.DataFrame(predictions)
predictions

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,1.098875e+06,1.179266e+06,1.230353e+06,1.248208e+06,1.249791e+06,1.230700e+06,1.216624e+06,1.220263e+06,1.234459e+06,1.194853e+06,...,1.035217e+06,1.011041e+06,1.006694e+06,9.948373e+05,9.809849e+05,971318.136213,964016.515370,968333.947820,966133.853318,9.894219e+05
1,1.127644e+06,1.202014e+06,1.258836e+06,1.285398e+06,1.282837e+06,1.272252e+06,1.277046e+06,1.262489e+06,1.224649e+06,1.181172e+06,...,1.025550e+06,1.008903e+06,1.002323e+06,9.908974e+05,9.800246e+05,966874.620905,960130.127361,955885.103753,949722.337785,9.544342e+05
2,1.029083e+06,1.091642e+06,1.136000e+06,1.150227e+06,1.150839e+06,1.139192e+06,1.137999e+06,1.128872e+06,1.111646e+06,1.101194e+06,...,9.801146e+05,9.717538e+05,9.843176e+05,9.702174e+05,9.614549e+05,954328.464845,953482.285325,958791.425179,963228.661256,1.008807e+06
3,9.762354e+05,1.007240e+06,1.025810e+06,1.038248e+06,1.040591e+06,1.029714e+06,1.017665e+06,1.004718e+06,9.914921e+05,9.911053e+05,...,9.974661e+05,1.000210e+06,9.984656e+05,9.869491e+05,9.745344e+05,966533.183316,962480.593121,969207.200816,973923.601687,1.015822e+06
4,1.093520e+06,1.174951e+06,1.224610e+06,1.241979e+06,1.241011e+06,1.230385e+06,1.225250e+06,1.205392e+06,1.163575e+06,1.142256e+06,...,1.053206e+06,1.030603e+06,1.029370e+06,1.014193e+06,1.003928e+06,994187.446192,988809.249866,993782.458884,998325.660601,1.041605e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,1.120868e+06,1.192846e+06,1.239101e+06,1.253733e+06,1.247064e+06,1.240991e+06,1.244748e+06,1.243440e+06,1.237939e+06,1.211036e+06,...,1.051904e+06,1.026033e+06,1.009506e+06,9.932342e+05,9.854765e+05,972747.938466,971272.874899,974813.194914,981866.580420,9.924786e+05
135,1.063431e+06,1.122428e+06,1.164584e+06,1.179453e+06,1.174663e+06,1.164487e+06,1.167216e+06,1.156606e+06,1.143571e+06,1.131235e+06,...,1.033281e+06,1.004117e+06,9.861997e+05,9.739993e+05,9.654283e+05,959443.606251,960836.754035,966929.252934,984725.494156,1.032332e+06
136,1.011812e+06,1.038054e+06,1.063963e+06,1.080658e+06,1.082906e+06,1.081560e+06,1.073858e+06,1.067566e+06,1.063752e+06,1.061575e+06,...,1.047069e+06,1.016271e+06,9.991060e+05,9.819875e+05,9.743767e+05,965323.930804,963147.050268,965727.279343,971115.219161,9.827651e+05
137,1.108458e+06,1.177679e+06,1.220475e+06,1.236528e+06,1.234166e+06,1.224439e+06,1.228260e+06,1.221840e+06,1.208163e+06,1.187769e+06,...,1.002887e+06,9.765492e+05,9.681787e+05,9.541998e+05,9.474083e+05,942077.656230,943681.384484,951720.564829,974558.314850,1.026580e+06


In [23]:
actuals = []
for i in range(NUM_OF_MODELS):
    y_columns = y_cols(0)
    valid_y = validation[y_columns].values
    actuals.append(valid_y)

In [24]:
actuals = pd.DataFrame(actuals).T
actuals

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,...,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06,1.099876e+06
1,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,...,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06,1.127879e+06
2,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,...,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06,1.029596e+06
3,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,...,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05,9.764967e+05
4,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,...,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06,1.093281e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,...,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06,1.120652e+06
135,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,...,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06,1.062749e+06
136,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,...,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06,1.012115e+06
137,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,...,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06,1.107557e+06


In [25]:
def absolute_errors(actual, predicted):
    return np.abs(actual - predicted)

def absolute_perc_errors(actual, predicted):
    return np.abs((actual - predicted) / actual) * 100

def squared_errors(actual, predicted):
    return (np.abs(actual - predicted))**2

In [26]:
aes = absolute_errors(actuals, predictions)
apes = absolute_perc_errors(actuals, predictions)
ses = squared_errors(actuals, predictions)

In [27]:
mae = aes.mean().mean()
mae

84468.70415298537

In [28]:
mape = apes.mean().mean()
mape

7.809259421111075

In [29]:
rmse = ses.mean().mean()
rmse

10021283206.214373